In [1]:
import sys
sys.path.append('..')
import processing
import configs.common as cc

loader = processing.DatasetLoader('E:\\GitHub\\dataset\\np_dataset')
train_dataloader, test_dataloader = loader.get_dataloaders()
# random_sample = loader.get_random_sample('train')
# random_sample
for src, trg, meta in train_dataloader:
    break
# src, trg, meta = random_sample

In [2]:
from train import new_model
import torch
type = 'xlstm'
model_name = 'loss_0.01_time_2025-04-26-09-44-39.pth'
model = new_model(type)
model.load_state_dict(torch.load(f'../pretrained/{type}/{model_name}'))
model.to('cuda')

xLSTM(
  (token_embedding_table): Embedding(963, 512)
  (metadata_embedding_table): Embedding(154, 512)
  (layers): ModuleList(
    (0): sLSTMblock(
      (ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (conv): CausalConv1D(
        (conv): Conv1d(512, 512, kernel_size=(64,), stride=(1,), padding=(63,))
      )
      (drop): Dropout(p=0.01, inplace=False)
      (i_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
        )
      )
      (f_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
        )
      )
      (o_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
        )
      )
      (z_gate): BlockDiagonal(
        (blocks): ModuleList(
          (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
        )
      )
      (ri_gate):

In [ ]:
def generate(model, context_len, token_ids, meta_ids, num_tokens=1000, device='cpu'):
    model.eval()
    generated = token_ids.detach().cpu().numpy().tolist()[0]

    with torch.no_grad():
            for _ in range(num_tokens):
                print(_, end="\r")
                logits = model(token_ids, meta_ids)  # (1, seq_len, vocab_size)
                logits_last = logits[:, -1, :]       # (1, vocab_size)
                probs = torch.softmax(logits_last, dim=-1)  # (1, vocab_size)

                # Get top-5
                topk_probs, topk_indices = torch.topk(probs, 2, dim=-1)  # Each: shape (1, 5)

                # Normalize to sum to 1
                topk_probs = topk_probs / topk_probs.sum(dim=-1, keepdim=True)

                # Sample
                next_token_idx = torch.multinomial(topk_probs, num_samples=1)  # (1, 1), value in [0..4]
                next_token = topk_indices.gather(1, next_token_idx)

                generated.append(next_token.item())

                token_ids = torch.cat([token_ids, next_token], dim=1)
                token_ids = token_ids[:, -context_len:]

            # If you want metadata, sample or set as zeros:
            # e.g., meta_ids = torch.cat([meta_ids, new_meta], dim=1)[:, -context_len:]

    return generated

new_seq = generate(model, cc.config.values.block_len, src[0].unsqueeze(0), meta[0].unsqueeze(0), num_tokens=500, device='cuda')

In [ ]:
decoded_notes_old = processing.decode(src[0])
processing.note_to_midi(decoded_notes_old, "comparison.mid")

decoded_notes_new = processing.decode(new_seq)
processing.note_to_midi(decoded_notes_new, "generated.mid")

In [3]:
import torch
import torch.nn.functional as F

def make_distributions():
    # Prepare output tensor
    block_len = cc.config.values.block_len
    device = cc.config.values.device
    vocab_size = cc.vocab_size
    distributions = torch.zeros(6, vocab_size, device=device)

    # For each token index, fill in regions with 1 as per your logic.
    start = [cc.start_idx["pitch"],
             cc.start_idx["dyn"],
             cc.start_idx["length"],
             cc.start_idx["time"],
             cc.start_idx["channel"],
             cc.start_idx["tempo"]]
    end   = [cc.start_idx["dyn"] - 1,
             cc.start_idx["length"] - 1,
             cc.start_idx["time"] - 1,
             cc.start_idx["channel"] - 1,
             cc.start_idx["tempo"] - 1,
             cc.vocab_size]   # block_len implies : to the end

    for token in range(6):
        if token == 0:
            distributions[token, start[1]:end[1]] = 1
        if token == 1:
            distributions[token, start[2]:end[2]] = 1
        if token == 2:
            distributions[token, start[3]:end[3]] = 1
            distributions[token, start[5]:end[5]] = 1
        if token == 3:
            distributions[token, start[5]:end[5]] = 1
        if token == 4:
            distributions[token, start[0]:end[0]] = 1
        if token == 5:
            distributions[token, start[4]:end[4]] = 1

    return distributions


def pick_distributions_by_prev_token(
        input_tokens: torch.Tensor
    ) -> torch.Tensor:
    boundaries = [
                cc.start_idx['dyn'] - 1,
                cc.start_idx['length'] - 1,
                cc.start_idx['time'] - 1,
                cc.start_idx['channel'] - 1,
                cc.start_idx['tempo'] - 1]
        
    bins = torch.tensor(boundaries, device=input_tokens.device)
        # Each token is assigned to a bucket 0..len(boundaries)
        # For example, with bins=[10,20,30]: 
        #   token <10 → 0, 10<=token<20 → 1, 20<=token<30 → 2, 30<=token → 3
    buckets = torch.bucketize(input_tokens, bins, right=False)
    distributions = make_distributions()

    # Ensure buckets is long (int64) and on the same device as distributions
    buckets = buckets.long().to(distributions.device)

    # Now use advanced indexing to get values
    output = distributions[buckets]  # shape: [6, 963]

    return output

def cross_entropy_loss(input, output, target, reduction='mean'):
    weights = pick_distributions_by_prev_token(input)
    log_probs = F.log_softmax(output, dim=1)
    loss = log_probs * weights 
    loss = -log_probs.gather(1, target.unsqueeze(1)).squeeze(1)
    if reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    else: # 'none'
        return loss

In [4]:
output = model(src, meta)

In [5]:
weights = pick_distributions_by_prev_token(src)
log_probs = F.log_softmax(output, dim=1)
# loss = log_probs * weights 
# loss = -log_probs.gather(1, trg.unsqueeze(1)).squeeze(1)

In [ ]:
import train
train.train(model)

Training started!
Epoch [1/1000], Step [10/362], Loss: 6.5125
Epoch [1/1000], Step [20/362], Loss: 5.6272
Epoch [1/1000], Step [30/362], Loss: 5.8188
